# Preliminary K-Means Clustering Scripts

Author: Brandon Fan

Last Edit Date: 12/28/17

In [ ]:
%matplotlib inline

In [ ]:
import sys
from importlib import reload
sys.path.insert(0, '../server')
from similarity_functions import Similarity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin

## Load Data & Construct Corpus

In [ ]:
# declare constants
BIBLE_FILE = '../server/files/english-web-bible.json'
GLOVE_FILE = '../server/files/glove.6B.200d.txt'

In [ ]:
# create glove vectors
SIMILARITY = Similarity(BIBLE_FILE, GLOVE_FILE, initialize=False)
SIMILARITY.initialize(create_matrix=False)

In [ ]:
# get data from similarity initialization, 
# go to server/similarity_functions/similarity.py for more info
verse_data = SIMILARITY.verse_data
vectorized_corpus = np.array([verse['vector'] for vector in verse_data])
del SIMILARITY

## Implement Clustering

In [ ]:
CLUSTERS = 5 # hyperparameter to change and test

### Mini-Batch K-Means Model

In [ ]:
# initialize mini-batch kmeans model
mbk_clf = MiniBatchKMeans(n_clusters=CLUSTERS)

In [ ]:
%%time
mbk_predictions = mbk_clf.fit_predict(vectorized_corpus) # fit and predict cluster values

In [ ]:
# add predictions to verse_data
for index, prediction in enumerate(mbk_predictions):
    verse_data[index]['mbk_cluster'] = prediction

### K-Means Model

In [ ]:
clf = KMeans(n_clusters=CLUSTERS)

In [ ]:
%%time
predictions = clf.fit_predict(vectorized_corpus) # fit and predict cluster values

In [ ]:
# add predictions to verse_data
for index, prediction in enumerate(predictions):
    verse_data[index]['cluster'] = prediction

## Plot Results

In [ ]:
colors = ['#4EACC5', '#FF9C34', '#4E9A06']

k_means_cluster_centers = np.sort(clf.cluster_centers_, axis=0) # get cluster centers
mbk_means_cluster_centers = np.sort(mbk_clf.cluster_centers_, axis=0) # get cluster centers
k_means_labels = pairwise_distances_argmin(vectorized_corpus, k_means_cluster_centers) # find labels for data
mbk_means_labels = pairwise_distances_argmin(vectorized_corpus, mbk_means_cluster_centers) # find labels for data
order = pairwise_distances_argmin(k_means_cluster_centers,
                                  mbk_means_cluster_centers) 

In [ ]:
# KMeans
ax = fig.add_subplot(1, 3, 1)
for k, col in zip(range(CLUSTERS), colors):
    my_members = k_means_labels == k
    cluster_center = k_means_cluster_centers[k]
    ax.plot(X[my_members, 0], X[my_members, 1], 'w',
            markerfacecolor=col, marker='.')
    ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
            markeredgecolor='k', markersize=6)
ax.set_title('KMeans')
ax.set_xticks(())
ax.set_yticks(())
plt.text(-3.5, 1.8,  'inertia: %f' % (clf.inertia_))

# MiniBatchKMeans
ax = fig.add_subplot(1, 3, 2)
for k, col in zip(range(CLUSTERS), colors):
    my_members = mbk_means_labels == order[k]
    cluster_center = mbk_means_cluster_centers[order[k]]
    ax.plot(X[my_members, 0], X[my_members, 1], 'w',
            markerfacecolor=col, marker='.')
    ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
            markeredgecolor='k', markersize=6)
ax.set_title('MiniBatchKMeans')
ax.set_xticks(())
ax.set_yticks(())
plt.text(-3.5, 1.8, 'inertia: %f' % mbk_clf.inertia_)

# Initialise the different array to all False
different = (mbk_means_labels == 4)
ax = fig.add_subplot(1, 3, 3)

for k in range(CLUSTERS):
    different += ((k_means_labels == k) != (mbk_means_labels == order[k]))

identic = np.logical_not(different)
ax.plot(vectorized_corpus[identic, 0], vectorized_corpus[identic, 1], 'w',
        markerfacecolor='#bbbbbb', marker='.')
ax.plot(vectorized_corpus[different, 0], vectorized_corpus[different, 1], 'w',
        markerfacecolor='m', marker='.')
ax.set_title('Difference')
ax.set_xticks(())
ax.set_yticks(())

plt.show()